In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tables
import mplfinance as mpf
import my_afml_fncs as my_afml

In [2]:
volume_bars_df = pd.read_hdf("tick_data/bars/volume_bars.h5", key="volume_bars")
volume_bars_df.head()

,time,open,high,low,close,volume
0,2024-11-15 08:39:56.469400,5967.00,5968.00,5930.75,5937.00,100001
1,2024-11-15 13:33:55.913324,5937.25,5953.25,5936.25,5944.25,100003
2,2024-11-15 14:30:17.843538,5944.50,5947.75,5935.00,5936.75,100040
3,2024-11-15 14:43:36.170628,5936.75,5941.75,5929.25,5930.00,100037
4,2024-11-15 15:02:28.997004,5929.75,5940.50,5920.00,5920.25,100002


In [3]:
volume_bars_df['time']=pd.to_datetime(volume_bars_df['time'])
volume_bars_df.set_index('time',inplace=True)
vol=my_afml.get_daily_volatility(volume_bars_df.close)

In [4]:
vol.mean()

0.0037081056391128505

In [5]:
es_cusum_events=my_afml.getTEvents(volume_bars_df.close,vol.mean())

In [6]:
t1=volume_bars_df.close.index.searchsorted(es_cusum_events+pd.Timedelta(days=1))
t1=t1[t1<volume_bars_df.close.shape[0]]
t1=pd.Series(volume_bars_df.close.index[t1],index=es_cusum_events[:t1.shape[0]])

In [7]:
ptsl=1
trgt=pd.Series(0.1,index=es_cusum_events)
minRet=0.01
numThread=10
trigger=my_afml.getEvents(volume_bars_df.close,es_cusum_events,ptsl,trgt,minRet,numThread,t1)

2025-02-04 22:11:13.685461 100.0% applyPtSlOnT1 done after 0.0 minutes. Remaining 0.0 minutes.
